In [1]:
import pandas as pd
import pickle

In [2]:
file_path = "/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/plbart_base_embeddings/df_fe_embeded_by_line_plbart.pkl"
with open(file_path, "rb") as fh:
  df = pickle.load(fh)

/var/folders/d8/cr798cb57kzd5kz_r8g8jm380000gn/T/ipykernel_36393/207127160.py:3: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  df = pickle.load(fh)


In [3]:
df.head()

,sample_id,severity,method,label,embeded_sequence_sum,embeded_sequence_avg
0,12150,critical,public SystemDiagnosticsDTO createSystemDi...,1,"[[45.86921691894531, -159.51837158203125, 23.2...","[[0.7280828356742859, -2.5320377349853516, 0.3..."
1,10168,critical,protected void addRequiredAlertProperties(Se...,1,"[[3.429672956466675, -8.77796459197998, 4.4912...","[[0.6859346032142639, -1.755592942237854, 0.89..."
2,12783,critical,"@SuppressWarnings(""try"")\n private void...",1,"[[247.75167846679688, -129.33477783203125, -16...","[[2.0142412185668945, -1.0515022277832031, -0...."
3,12754,critical,public void validateDepositDetailForUpdate...,1,"[[65.13414001464844, -127.0123519897461, -15.7...","[[0.5815548300743103, -1.1340388059616089, -0...."
4,4277,critical,@Override\n public ListenableFuture<Voi...,1,"[[125.08989715576172, -107.79071044921875, 18....","[[1.6245441436767578, -1.3998793363571167, 0.2..."


In [4]:
embeded_sum = df.loc[:, ['sample_id', 'severity', 'label', 'embeded_sequence_sum']]
embeded_avg = df.loc[:, ['sample_id', 'severity', 'label', 'embeded_sequence_avg']]
embeded_sum.head()

,sample_id,severity,label,embeded_sequence_sum
0,12150,critical,1,"[[45.86921691894531, -159.51837158203125, 23.2..."
1,10168,critical,1,"[[3.429672956466675, -8.77796459197998, 4.4912..."
2,12783,critical,1,"[[247.75167846679688, -129.33477783203125, -16..."
3,12754,critical,1,"[[65.13414001464844, -127.0123519897461, -15.7..."
4,4277,critical,1,"[[125.08989715576172, -107.79071044921875, 18...."


In [5]:
print(embeded_sum["embeded_sequence_sum"].head())
print(embeded_sum["embeded_sequence_sum"].apply(type).value_counts())

0    [[45.86921691894531, -159.51837158203125, 23.2...
1    [[3.429672956466675, -8.77796459197998, 4.4912...
2    [[247.75167846679688, -129.33477783203125, -16...
3    [[65.13414001464844, -127.0123519897461, -15.7...
4    [[125.08989715576172, -107.79071044921875, 18....
Name: embeded_sequence_sum, dtype: object
embeded_sequence_sum
<class 'list'>    3027
Name: count, dtype: int64


In [6]:
def process_embedding(x):
    if x is None:
        return []  
    elif isinstance(x, pd.Series):
        return " ".join(map(str, x))
    else:
        return str(x)  

embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(process_embedding)


In [7]:
print(embeded_sum["embeded_sequence_sum"].head())

0    [[45.86921691894531, -159.51837158203125, 23.2...
1    [[3.429672956466675, -8.77796459197998, 4.4912...
2    [[247.75167846679688, -129.33477783203125, -16...
3    [[65.13414001464844, -127.0123519897461, -15.7...
4    [[125.08989715576172, -107.79071044921875, 18....
Name: embeded_sequence_sum, dtype: object


In [8]:
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(
    lambda x: " ".join(str(i) for i in x) if isinstance(x, pd.Series) else str(x)
)

In [9]:
import re

def parse_flat_embedding(embedding_str):
    # Ensure input is a string
    if not isinstance(embedding_str, str):
        return []
    try:
        # Extract numerical values from the string
        values = re.findall(r"[-+]?\d*\.\d+|\d+", embedding_str)
        return [float(v) for v in values]
    except Exception as e:
        print(f"Error parsing: {embedding_str} -> {e}")
        return []


In [10]:
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(parse_flat_embedding)

In [11]:
n = len(embeded_sum["embeded_sequence_sum"][0])
print(f"Embedding length: {n}")

embed_lengths = embeded_sum["embeded_sequence_sum"].apply(len)
length_counts = embed_lengths.value_counts()
most_common_length = length_counts.idxmax()

n = most_common_length
print(f"Most common sequence length: {most_common_length}")
print(f"Count of rows with this length: {length_counts[most_common_length]}")


Embedding length: 768
Most common sequence length: 768
Count of rows with this length: 3017


In [12]:
 # Function to handle the trimming or padding of the sequence
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(
    lambda x: (x[:n] if len(x) > n else x + [0] * (n - len(x))) if isinstance(x, list) else [0] * n
)

print(embeded_sum["embeded_sequence_sum"].head())

0    [45.86921691894531, -159.51837158203125, 23.20...
1    [3.429672956466675, -8.77796459197998, 4.49120...
2    [247.75167846679688, -129.33477783203125, -16....
3    [65.13414001464844, -127.0123519897461, -15.72...
4    [125.08989715576172, -107.79071044921875, 18.8...
Name: embeded_sequence_sum, dtype: object


In [13]:
embed_lengths = embeded_sum["embeded_sequence_sum"].apply(len)
print(embed_lengths.value_counts())


embeded_sequence_sum
768    3027
Name: count, dtype: int64


In [14]:
# Create column names for embeddings
columns = [f'emb_{i+1}' for i in range(n)]

# Create a DataFrame for embedding values
data = pd.DataFrame(embeded_sum["embeded_sequence_sum"].to_list(), columns=columns)

# Add metadata columns
data['label'] = embeded_sum['label']
data['sample_id'] = embeded_sum['sample_id']
data['severity'] = embeded_sum['severity']

# Display the result
print(data.head())


        emb_1       emb_2      emb_3       emb_4       emb_5       emb_6  \
0   45.869217 -159.518372  23.207367 -137.124603  128.811096  -82.769165   
1    3.429673   -8.777965   4.491210  -10.657660    9.625399   -4.418340   
2  247.751678 -129.334778 -16.591146 -173.292419  301.242645 -194.701111   
3   65.134140 -127.012352 -15.722034 -256.294617  304.530762 -158.203156   
4  125.089897 -107.790710  18.801786 -104.372971  193.609970 -143.885544   

        emb_7      emb_8       emb_9      emb_10  ...     emb_762     emb_763  \
0  -81.768135  -0.640440 -128.941055  -85.681137  ...   81.719536  111.293434   
1    3.721420  -7.408656   -4.963110  -15.298643  ...    7.573229    5.523479   
2 -162.284851  -9.360210 -156.371552 -213.367981  ...  124.247910   92.825050   
3 -216.614334  -7.443196 -264.936035 -183.173889  ...  113.942245   72.379677   
4  -98.533447  15.337610 -114.026794  -96.366798  ...   83.062164   54.470287   

      emb_764     emb_765     emb_766     emb_767     em

In [15]:
dataset_path = '/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/plbart_base_embeddings/processed_embeddings/plbart_embed_line_by_line_sum.pkl'
pd.to_pickle(data, dataset_path)

In [16]:
print(embeded_avg["embeded_sequence_avg"].head())
print(embeded_avg["embeded_sequence_avg"].apply(type).value_counts())

0    [[0.7280828356742859, -2.5320377349853516, 0.3...
1    [[0.6859346032142639, -1.755592942237854, 0.89...
2    [[2.0142412185668945, -1.0515022277832031, -0....
3    [[0.5815548300743103, -1.1340388059616089, -0....
4    [[1.6245441436767578, -1.3998793363571167, 0.2...
Name: embeded_sequence_avg, dtype: object
embeded_sequence_avg
<class 'list'>    3027
Name: count, dtype: int64


In [17]:
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(
    lambda x: " ".join(str(i) for i in x) if isinstance(x, pd.Series) else str(x)
)

In [18]:
n = len(embeded_avg["embeded_sequence_avg"][0])
print(f"Embedding length: {n}")

Embedding length: 15720


In [19]:
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(parse_flat_embedding)

In [20]:
embed_lengths = embeded_avg["embeded_sequence_avg"].apply(len)
length_counts = embed_lengths.value_counts()
most_common_length = length_counts.idxmax()

n = most_common_length
print(f"Most common sequence length: {most_common_length}")
print(f"Count of rows with this length: {length_counts[most_common_length]}")

Most common sequence length: 768
Count of rows with this length: 2937


In [21]:
# Function to handle the trimming or padding of the sequence
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(
    lambda x: (x[:n] if len(x) > n else x + [0] * (n - len(x))) if isinstance(x, list) else [0] * n
)

print(embeded_avg["embeded_sequence_avg"].head())

0    [0.7280828356742859, -2.5320377349853516, 0.36...
1    [0.6859346032142639, -1.755592942237854, 0.898...
2    [2.0142412185668945, -1.0515022277832031, -0.1...
3    [0.5815548300743103, -1.1340388059616089, -0.1...
4    [1.6245441436767578, -1.3998793363571167, 0.24...
Name: embeded_sequence_avg, dtype: object


In [22]:
embed_lengths = embeded_avg["embeded_sequence_avg"].apply(len)
print(embed_lengths.value_counts())

embeded_sequence_avg
768    3027
Name: count, dtype: int64


In [23]:
columns = [f'emb_{i+1}' for i in range(n)]
data = pd.DataFrame(embeded_avg["embeded_sequence_avg"].to_list(), columns=columns)
data['label'] = embeded_avg['label']
data['sample_id'] = embeded_avg['sample_id']
data['severity'] = embeded_avg['severity']
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,emb_768,label,sample_id,severity
0,0.728083,-2.532038,0.368371,-2.176581,2.044621,-1.313796,-1.297907,-0.010166,-2.046683,-1.360018,...,1.297135,1.766562,-1.514855,-3.513744,-3.533521,1.863348,-0.564521,1,12150,critical
1,0.685935,-1.755593,0.898242,-2.131532,1.925080,-0.883668,0.744284,-1.481731,-0.992622,-3.059729,...,1.514646,1.104696,-1.490182,-3.218683,-2.581105,2.782470,1.722228,1,10168,critical
2,2.014241,-1.051502,-0.134887,-1.408881,2.449127,-1.582936,-1.319389,-0.076099,-1.271313,-1.734699,...,1.010146,0.754675,-1.005410,-2.877481,-2.725385,2.086285,0.148330,1,12783,critical
3,0.581555,-1.134039,-0.140375,-2.288345,2.719025,-1.412528,-1.934057,-0.066457,-2.365500,-1.635481,...,1.017341,0.646247,-0.976179,-2.642240,-2.973845,2.562416,-0.896226,1,12754,critical
4,1.624544,-1.399879,0.244179,-1.355493,2.514415,-1.868643,-1.279655,0.199190,-1.480868,-1.251517,...,1.078729,0.707406,-1.293822,-3.561596,-2.398942,1.914786,0.081015,1,4277,critical


In [24]:
dataset_path = '/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/plbart_base_embeddings/processed_embeddings/plbart_embed_line_by_line_avg.pkl'
pd.to_pickle(data, dataset_path)